In [ ]:
import requests
import bs4

from bs4 import BeautifulSoup

import pandas as pd

In [ ]:
#url = 'https://catalog.siena.edu/content.php?catoid=18&navoid=656'

# But we downloaded by hand

infilename = 'catalog_source.html'
infile = open(infilename)


page = infile.read()
#print(page)

soup = BeautifulSoup(page)



In [ ]:
#soup

In [ ]:
#soup.body

for a in soup.find_all('a', href=True):
    if a['href'].find('preview_program')>=0:
        print("Found the URL:", a['href'])



In [ ]:
files = []

tables = soup.body.find_all('table')

for table in tables:
    #print(table)
    #print()
    
    entry = table.find_all('td')
    for t in entry:
        values = t.find_all('a')
        for v in values:
            #print(v)
            text = str(v)
            #print(text)
            if text.find('href')>=0:
                #print(text)
                file = text.split('href="')[-1].split('"')[0]
                if file.find('program_preview')>=0 or file.find('csv')>=0:  
                    files.append(file)


In [ ]:
url = 'https://catalog.siena.edu/content.php?catoid=18&navoid=656'

dfs = pd.read_html(url)

dfs


In [ ]:
dfs[4]

In [ ]:
for i,df in enumerate(dfs):
    print(f'{i} --------------------------------')
    print(df)
    print()
    print()

# Parsing department pages


In [ ]:
infilename = 'catalog_entry_physics.html'

infile = open(infilename)


page = infile.read()
#print(page)


#soup = BeautifulSoup(page, 'lxml')
soup = BeautifulSoup(page, 'html.parser')


#soup = soup.prettify()



In [ ]:
soup

In [ ]:
#soup.find_all("div", class_="acalog-core")

soup.find_all("div", {"class":"acalog-core"})

In [ ]:
#soup

In [ ]:
# Physics
url = 'https://catalog.siena.edu/preview_program.php?catoid=18&poid=2806&returnto=656'

r = requests.get(url)

soup = BeautifulSoup(r.content, "lxml")

data = soup.find_all("div", {"class": "acalog-core"})


In [ ]:
for d in data:
    print(d)
    print("\n-----------------------------------\n")

In [ ]:
d = data[0]
d

In [ ]:
type(d)

In [ ]:
d.text

In [ ]:
type(d.contents)

In [ ]:
d.contents[0].text

In [ ]:
d.contents[0]

In [ ]:
d.contents[0].find_all('a')

In [ ]:
for i,d in enumerate(data):
    #x = d.contents[0].text
    print(f'{i} -----------')
    for con in d.contents:
        x = con.text
        #print(type(x))
        print(x)
        print()

In [ ]:
d.contents[2].findAll('a')

In [ ]:
d.contents[2].text

In [ ]:

'''
for i,d in enumerate(data):
    #x = d.contents[0].text
    print(f'{i} -----------')
    for con in d.contents:
        x = con.text
        #print(type(x))
        print(x)
        print()
''';

text = data[0].contents[0].text

print(text)

program = text.split('Requirements for the ')[1].split('(')[0]
print(program)



In [ ]:
# Use the h2 tag to find when credits are listed

def extract_info(data):
    
    program_name = None
    program_type = None
    major_credits = []
    aux_credits = None
    major_or_minor = None
    
    for d in data:
    
        h2s = d.find_all('h2')
        if len(h2s) == 1:

            # Most every program has the Auxilary as h2 header, but not some of them 
            # like American studies who has it as h3 (shrug)
            texts = h2s[0].contents
            for text in texts:
                # text is a bs4.element.tag, not a string
                #print(text)
                #print(type(text))
                #print(text.find("Requirements"))
                if str(text).find("Requirements for")>=0:
                    print(str(text))
                    print()
                    #program_name = text.split(' in ')[1].split(' ')[0]
                    program_name = None#text.split('Requirements for the ')[1].split('(')[0]
                    major_credits.append(text.split('(')[-1].split(' ')[0])
                    major_or_minor = None
                    if str(text).find('Major')>=0:
                        major_or_minor = 'Major'
                    elif str(text).find('Minor')>=0:
                        major_or_minor = 'Minor'

                    program_type = None
                    if str(text).find('B.S.')>=0:
                        program_type = 'B.S.'
                    elif str(text).find('B.A.')>=0:
                        program_type = 'B.A.'
    
                elif str(text).find("Auxiliary ")>=0:
                    print()
                    print(text)
                    print()
                    aux_credits = text.split('(')[1].split(' ')[0]

    return program_name,  major_or_minor, program_type, major_credits, aux_credits 
    

program_name,  major_or_minor, program_type, major_credits, aux_credits = extract_info(data)
    
print(program_name)
print(major_or_minor)
print(program_type)
print(major_credits)
print(aux_credits)


In [ ]:
text

In [ ]:
str(text).find("Requirements")

# This seems to be working!

Note that I started by downloading the HTML for the listing of the "Academic Programs" and saved it as `catalog_source.html`.

In [ ]:
# Try to scrape many more

mydict = {}
mydict['Program name'] = []
mydict['Major/Minor'] = []
mydict['Program type'] = []
mydict['Major credits'] = []
mydict['Auxiliary credits'] = []
mydict['Total credits'] = []

mydict['Major credits (lo)'] = []
mydict['Major credits (hi)'] = []

mydict['Auxiliary credits (lo)'] = []
mydict['Auxiliary credits (hi)'] = []


infilename = 'catalog_source.html'
infile = open(infilename)


page = infile.read()
#print(page)

soup = BeautifulSoup(page)

MAX_PAGES = 30000

icount = 0
for a in soup.find_all('a', href=True):
    if a['href'].find('preview_program')>=0:
        print("Found the URL:", a['href'])

        #url = 'https://catalog.siena.edu/preview_program.php?catoid=18&poid=2806&returnto=656'
        url = a['href']
        
        r = requests.get(url)
        soup_program = BeautifulSoup(r.content, "lxml")
        data = soup_program.find_all("div", {"class": "acalog-core"})

        program_name,  major_or_minor, program_type, major_credits, aux_credits = extract_info(data)
        
        program_name = soup_program.find_all('h1')[0].text

        #x[0].text
            
        print(program_name)
        print(major_or_minor)
        print(program_type)
        print(major_credits)
        print(aux_credits)

        mydict['Program name'].append(program_name)
        mydict['Major/Minor'].append(major_or_minor)
        mydict['Program type'].append(program_type)

        credlo = 0
        credhi = 0
        cred = 0
        for c in major_credits:
            if c.find('-')>=0:
                clo = c.split('-')[0]
                chi = c.split('-')[1]
                if clo.isnumeric() and chi.isnumeric():
                    clo = int(clo)
                    chi = int(chi)
                else:
                    clo,chi = 0,0
            else:
                if c.isnumeric() is False:
                    c = '0'
                clo = int(c)
                chi = int(c)
            credlo += clo
            credhi += chi
        cred = (credlo+credhi)/2

        aclo=0
        achi=0
        acredlo = 0
        acredhi = 0
        acred= 0
        if aux_credits is None:
            aux_credits = '0'
        c = aux_credits
        if c.find('-')>=0:
            aclo = int(c.split('-')[0])
            achi = int(c.split('-')[1])
        else:
            aclo = int(c)
            achi = int(c)
        acredlo += aclo
        acredhi += achi
        acred = (acredlo+acredhi)/2

        total_credits = acred + cred

        
        mydict['Major credits'].append(cred)
        mydict['Auxiliary credits'].append(acred)
        
        mydict['Major credits (lo)'].append(credlo)
        mydict['Major credits (hi)'].append(credhi)
        
        mydict['Auxiliary credits (lo)'].append(acredlo)
        mydict['Auxiliary credits (hi)'].append(acredhi)

        mydict['Total credits'].append(total_credits)
        

        icount += 1

    if icount>=MAX_PAGES:
        break


In [ ]:
for key,val in mydict.items():
    print(key,len(val))

In [ ]:
print(mydict['Program name'])
print()
print(mydict['Major credits'])

In [ ]:
df = pd.DataFrame.from_dict(mydict)

df

In [ ]:
df.to_csv('programs.csv')

In [ ]:
# Get the program name
x = soup_program.find_all('h1')

x[0].text